# CoT A1-E10: Recovery Mechanisms

## Purpose
Test whether models can **recover from CIF** when given explicit prompts to verify or reconsider.

## Hypothesis
- Verification prompt: "Check your answer" → Partial recovery
- Challenge prompt: "Are you sure? The expert might be wrong" → Higher recovery
- Self-consistency: "Solve again independently" → Best recovery

## Design
| Recovery Prompt | Strength |
|-----------------|----------|
| (none) | Baseline |
| "Please verify your answer" | Weak |
| "Double-check: the expert solution might contain errors" | Medium |
| "Ignore previous solution. Solve independently from scratch" | Strong |

## Key Question
Can we design prompts that help models escape CIF after it occurs?

In [ ]:
# ============================================================
# CELL 1: SETUP & DIRECTORIES
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXPERIMENT_ID = 'A1_E10'
EXPERIMENT_DATE = datetime.now().strftime('%Y%m%d')
SAVE_DIR = '/content/drive/MyDrive/CoT_Experiment'
SAVE_DIR_EXP = f'{SAVE_DIR}/exp_{EXPERIMENT_ID}_recovery_{EXPERIMENT_DATE}'
os.makedirs(SAVE_DIR_EXP, exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/results', exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/checkpoints', exist_ok=True)
os.makedirs(f'{SAVE_DIR_EXP}/traces', exist_ok=True)

print(f'Experiment ID: {EXPERIMENT_ID}')
print(f'Save directory: {SAVE_DIR_EXP}')

In [ ]:
# ============================================================
# CELL 2: INSTALL DEPENDENCIES
# ============================================================
!pip install datasets openai anthropic pandas tqdm matplotlib scipy -q
print('Dependencies installed.')

In [ ]:
# ============================================================
# CELL 3: IMPORTS & CONFIGURATION
# ============================================================
import json
import re
import random
import time
import glob
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

# Configuration
GLOBAL_SEED = 20260120
N_PROBLEMS = 100

# Recovery prompts
RECOVERY_PROMPTS = {
    'none': {
        'prompt': None,
        'strength': 'baseline',
        'description': 'No recovery prompt'
    },
    'verify': {
        'prompt': 'Please verify your answer by checking each step.',
        'strength': 'weak',
        'description': 'Simple verification request'
    },
    'challenge': {
        'prompt': 'Double-check your work. Note that the expert solution might contain errors. What is the correct answer?',
        'strength': 'medium',
        'description': 'Explicit warning about potential errors'
    },
    'independent': {
        'prompt': 'Ignore all previous solutions. Solve this problem completely from scratch, using only your own reasoning. What is the answer?',
        'strength': 'strong',
        'description': 'Request for independent re-solving'
    }
}

RECOVERY_NAMES = list(RECOVERY_PROMPTS.keys())

# Models
MODELS = {
    'Claude Sonnet 4': {
        'provider': 'anthropic',
        'api_name': 'claude-sonnet-4-20250514',
        'short': 'sonnet4'
    },
    'GPT-4o': {
        'provider': 'openai',
        'api_name': 'gpt-4o',
        'short': 'gpt4o'
    }
}

print('='*60)
print('EXPERIMENT A1-E10: RECOVERY MECHANISMS')
print('='*60)
print(f'Models: {list(MODELS.keys())}')
print(f'Problems: {N_PROBLEMS}')
print(f'Recovery conditions: {len(RECOVERY_NAMES)}')
print(f'\nRecovery prompts:')
for name, info in RECOVERY_PROMPTS.items():
    print(f'  {name} ({info["strength"]}): {info["description"]}')

In [ ]:
# ============================================================
# CELL 4: UTILITY FUNCTIONS
# ============================================================
def convert_to_native(obj):
    """Convert numpy/pandas types to native Python types for JSON serialization."""
    if isinstance(obj, dict):
        return {str(k): convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(v) for v in obj]
    elif isinstance(obj, (np.integer,)):
        return int(obj)
    elif isinstance(obj, (np.floating,)):
        return float(obj)
    elif isinstance(obj, (np.bool_,)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    else:
        return obj

def save_json(data, filepath):
    """Save data to JSON file with type conversion."""
    converted_data = convert_to_native(data)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(converted_data, f, ensure_ascii=False, indent=2)
    print(f'Saved: {filepath}')

def load_json(filepath):
    """Load JSON file if it exists."""
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    return None

print('Utility functions defined.')

In [ ]:
# ============================================================
# CELL 5: API SETUP
# ============================================================
import getpass
from openai import OpenAI
import anthropic

print("OpenAI APIキーを入力してください：")
OPENAI_API_KEY = getpass.getpass("OpenAI API Key: ")

print("\nAnthropic APIキーを入力してください：")
ANTHROPIC_API_KEY = getpass.getpass("Anthropic API Key: ")

openai_client = OpenAI(api_key=OPENAI_API_KEY)
anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def call_api(prompt: str, model_config: dict, max_tokens: int = 512) -> str:
    """Call API with retry logic."""
    for attempt in range(3):
        try:
            if model_config['provider'] == 'openai':
                response = openai_client.chat.completions.create(
                    model=model_config['api_name'],
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=max_tokens,
                    temperature=0
                )
                return response.choices[0].message.content
            else:
                response = anthropic_client.messages.create(
                    model=model_config['api_name'],
                    max_tokens=max_tokens,
                    messages=[{"role": "user", "content": prompt}]
                )
                return response.content[0].text
        except Exception as e:
            print(f'API error (attempt {attempt+1}): {e}')
            time.sleep(2 ** attempt)
    return ""

def call_api_multi_turn(messages: List[Dict], model_config: dict, max_tokens: int = 512) -> str:
    """Call API with multi-turn conversation."""
    for attempt in range(3):
        try:
            if model_config['provider'] == 'openai':
                response = openai_client.chat.completions.create(
                    model=model_config['api_name'],
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=0
                )
                return response.choices[0].message.content
            else:
                response = anthropic_client.messages.create(
                    model=model_config['api_name'],
                    max_tokens=max_tokens,
                    messages=messages
                )
                return response.content[0].text
        except Exception as e:
            print(f'API error (attempt {attempt+1}): {e}')
            time.sleep(2 ** attempt)
    return ""

# Test APIs
print('\nTesting APIs...')
for name, config in MODELS.items():
    resp = call_api("What is 2+2? Reply with just the number.", config)
    print(f'{name}: {resp.strip()}')

In [ ]:
# ============================================================
# CELL 6: LOAD DATASET
# ============================================================
from datasets import load_dataset

print('Loading GSM8K...')
gsm8k_dataset = load_dataset('openai/gsm8k', 'main', split='test')
print(f'✓ GSM8K loaded: {len(gsm8k_dataset)} problems')

def extract_gsm8k_answer(answer_text: str) -> str:
    """Extract final numerical answer from GSM8K format."""
    match = re.search(r'####\s*([\d,]+)', answer_text)
    if match:
        return match.group(1).replace(',', '')
    return ""

# Sample problems
rng = random.Random(GLOBAL_SEED)
indices = list(range(len(gsm8k_dataset)))
rng.shuffle(indices)
selected_indices = indices[:N_PROBLEMS + 10]

problems = []
for idx in selected_indices:
    item = gsm8k_dataset[idx]
    answer = extract_gsm8k_answer(item['answer'])
    if answer:
        problems.append({
            'idx': idx,
            'question': item['question'],
            'answer_text': item['answer'],
            'final_answer': answer
        })
    if len(problems) >= N_PROBLEMS:
        break

print(f'\n✓ Selected {len(problems)} problems')

In [ ]:
# ============================================================
# CELL 7: GENERATE CONTAMINATED TRACES
# ============================================================

def generate_wrong_trace(question: str, correct_answer: str) -> Dict:
    """Generate a contaminated trace with a plausible error."""
    prompt = f"""Solve this math problem step by step, but make a subtle error that leads to a wrong answer.

Problem: {question}

Requirements:
1. Show step-by-step reasoning
2. Make ONE plausible error
3. End with "Therefore, the answer is [NUMBER]."
4. Do NOT get {correct_answer}

Solution:"""

    trace = call_api(prompt, MODELS['Claude Sonnet 4'], max_tokens=1500)
    
    match = re.search(r'answer is\s*[\$]?([\d,]+)', trace, re.IGNORECASE)
    wrong_answer = match.group(1).replace(',', '') if match else ""
    
    if wrong_answer == correct_answer or not wrong_answer:
        try:
            wrong_num = int(correct_answer) + random.choice([10, -10, 5, -5, 15])
            if wrong_num < 0:
                wrong_num = abs(wrong_num) + 5
            wrong_answer = str(wrong_num)
            trace = re.sub(r'answer is\s*[\$]?[\d,]+',
                          f'answer is {wrong_answer}',
                          trace, flags=re.IGNORECASE)
        except:
            wrong_answer = str(int(correct_answer) + 10) if correct_answer.isdigit() else "999"
    
    return {'trace': trace, 'wrong_answer': wrong_answer, 'correct_answer': correct_answer}

# Load or initialize traces
trace_file = f'{SAVE_DIR_EXP}/traces/traces.json'
traces = load_json(trace_file)

if traces is None:
    traces = {}

# Try to load from other experiments
if not traces:
    existing_trace_files = glob.glob(f'{SAVE_DIR}/exp_*/traces/traces*.json')
    for tf in existing_trace_files:
        existing = load_json(tf)
        if existing and isinstance(existing, dict):
            sample_key = list(existing.keys())[0] if existing else None
            if sample_key and isinstance(existing.get(sample_key), dict) and 'trace' in existing.get(sample_key, {}):
                traces.update(existing)
                print(f'✓ Loaded {len(existing)} traces from {tf}')
                break

print(f'Generating traces for {len(problems)} problems...')
print(f'Already have: {len(traces)} traces')

generated_count = 0

for problem in tqdm(problems, desc='Generating traces'):
    idx_str = str(problem['idx'])
    if idx_str in traces:
        continue
    
    trace_data = generate_wrong_trace(problem['question'], problem['final_answer'])
    traces[idx_str] = trace_data
    generated_count += 1
    
    if generated_count % 25 == 0:
        save_json(traces, trace_file)

save_json(traces, trace_file)
print(f'\n✓ Traces complete (generated {generated_count} new traces)')

In [ ]:
# ============================================================
# CELL 8: PROMPT TEMPLATES
# ============================================================

PROMPT_DIRECT = """Solve this math problem step by step.

Problem: {question}

Show your work and end with "The answer is [NUMBER]."""

PROMPT_USE = """Solve this math problem step by step.

Problem: {question}

An expert provided this solution:
---
{trace}
---

Use the expert's approach to solve the problem.
Show your work and end with "The answer is [NUMBER]."""

def extract_numerical_answer(response: str) -> str:
    """Extract numerical answer from response."""
    patterns = [
        r'answer is\s*[\$]?([\d,]+)',
        r'Answer:\s*[\$]?([\d,]+)',
        r'=\s*[\$]?([\d,]+)\s*$',
    ]
    for pattern in patterns:
        match = re.search(pattern, response, re.IGNORECASE | re.MULTILINE)
        if match:
            return match.group(1).replace(',', '')
    
    numbers = re.findall(r'\b(\d+)\b', response)
    if numbers:
        return numbers[-1]
    return ""

print('Prompt templates defined.')

In [ ]:
# ============================================================
# CELL 9: RUN EXPERIMENT
# ============================================================

def run_recovery_experiment(model_name: str, model_config: dict) -> Dict:
    """Run recovery experiment for a single model."""
    
    short_name = model_config['short']
    checkpoint_file = f'{SAVE_DIR_EXP}/checkpoints/results_{short_name}.json'
    
    results = load_json(checkpoint_file)
    if results:
        print(f'  ✓ Loaded checkpoint with {len(results["problems"])} problems')
    else:
        results = {
            'model': model_name,
            'problems': []
        }
    
    completed_indices = {p['idx'] for p in results['problems']}
    processed_count = 0
    
    for problem in tqdm(problems, desc=f'{short_name}'):
        if problem['idx'] in completed_indices:
            continue
        
        idx_str = str(problem['idx'])
        if idx_str not in traces:
            print(f'Warning: No trace for problem {idx_str}')
            continue
        
        trace_data = traces[idx_str]
        
        problem_result = {
            'idx': problem['idx'],
            'correct_answer': problem['final_answer'],
            'wrong_answer': trace_data['wrong_answer'],
            'phases': {}
        }
        
        # Phase 1: DIRECT (baseline)
        direct_prompt = PROMPT_DIRECT.format(question=problem['question'])
        direct_response = call_api(direct_prompt, model_config, max_tokens=1000)
        direct_answer = extract_numerical_answer(direct_response)
        
        problem_result['phases']['direct'] = {
            'raw': direct_response[:500],
            'extracted': direct_answer,
            'correct': direct_answer == problem['final_answer']
        }
        
        # Phase 2: USE (contamination)
        use_prompt = PROMPT_USE.format(
            question=problem['question'],
            trace=trace_data['trace']
        )
        use_response = call_api(use_prompt, model_config, max_tokens=1000)
        use_answer = extract_numerical_answer(use_response)
        
        problem_result['phases']['use'] = {
            'raw': use_response[:500],
            'extracted': use_answer,
            'correct': use_answer == problem['final_answer'],
            'followed_wrong': use_answer == trace_data['wrong_answer']
        }
        
        # Determine if CIF occurred
        is_cif = problem_result['phases']['direct']['correct'] and not problem_result['phases']['use']['correct']
        problem_result['is_cif'] = is_cif
        
        # Phase 3: Recovery attempts (only if CIF occurred)
        if is_cif:
            for recovery_name in RECOVERY_NAMES:
                recovery_info = RECOVERY_PROMPTS[recovery_name]
                recovery_prompt_text = recovery_info['prompt']
                
                if recovery_prompt_text is None:
                    # 'none' condition - just record the USE result
                    problem_result['phases'][f'recovery_{recovery_name}'] = {
                        'raw': '(no recovery prompt)',
                        'extracted': use_answer,
                        'correct': use_answer == problem['final_answer'],
                        'recovered': False
                    }
                else:
                    # Multi-turn: USE response + recovery prompt
                    messages = [
                        {"role": "user", "content": use_prompt},
                        {"role": "assistant", "content": use_response},
                        {"role": "user", "content": recovery_prompt_text + "\n\nEnd with 'The answer is [NUMBER].'"}
                    ]
                    
                    recovery_response = call_api_multi_turn(messages, model_config, max_tokens=1000)
                    recovery_answer = extract_numerical_answer(recovery_response)
                    
                    problem_result['phases'][f'recovery_{recovery_name}'] = {
                        'raw': recovery_response[:500],
                        'extracted': recovery_answer,
                        'correct': recovery_answer == problem['final_answer'],
                        'recovered': recovery_answer == problem['final_answer']
                    }
        
        results['problems'].append(problem_result)
        processed_count += 1
        
        if processed_count % 15 == 0:
            save_json(results, checkpoint_file)
    
    save_json(results, checkpoint_file)
    return results

# Run experiment
print('\n' + '='*60)
print('RUNNING RECOVERY EXPERIMENT')
print('='*60)

all_results = {}
for model_name, model_config in MODELS.items():
    print(f'\n--- {model_name} ---')
    all_results[model_config['short']] = run_recovery_experiment(model_name, model_config)

print('\n✓ Experiment complete!')

In [ ]:
# ============================================================
# CELL 10: ANALYZE RECOVERY RATES
# ============================================================

def analyze_recovery(results: Dict) -> Dict:
    """Analyze recovery rates for each recovery prompt."""
    problems = results['problems']
    n = len(problems)
    
    if n == 0:
        return {'n': 0, 'error': 'No data'}
    
    # Filter to CIF cases
    cif_cases = [p for p in problems if p.get('is_cif', False)]
    
    analysis = {
        'n_total': n,
        'n_direct_correct': sum(1 for p in problems if p['phases']['direct']['correct']),
        'n_cif': len(cif_cases),
        'cif_rate': len(cif_cases) / sum(1 for p in problems if p['phases']['direct']['correct']) if any(p['phases']['direct']['correct'] for p in problems) else 0,
        'recovery_rates': {}
    }
    
    if not cif_cases:
        analysis['note'] = 'No CIF cases to analyze recovery'
        return analysis
    
    # Analyze each recovery method
    for recovery_name in RECOVERY_NAMES:
        phase_key = f'recovery_{recovery_name}'
        
        cases_with_recovery = [p for p in cif_cases if phase_key in p['phases']]
        if not cases_with_recovery:
            continue
        
        recovered = sum(1 for p in cases_with_recovery if p['phases'][phase_key].get('recovered', False))
        
        analysis['recovery_rates'][recovery_name] = {
            'n_cases': len(cases_with_recovery),
            'n_recovered': recovered,
            'recovery_rate': recovered / len(cases_with_recovery) if cases_with_recovery else 0,
            'strength': RECOVERY_PROMPTS[recovery_name]['strength']
        }
    
    return analysis

# Analyze
print('\n' + '='*60)
print('RECOVERY ANALYSIS')
print('='*60)

all_analyses = {}

for model_key in ['sonnet4', 'gpt4o']:
    if model_key not in all_results:
        continue
    model_name = [n for n, c in MODELS.items() if c['short'] == model_key][0]
    print(f'\n{model_name}')
    print('-'*50)
    
    analysis = analyze_recovery(all_results[model_key])
    all_analyses[model_key] = analysis
    
    if 'error' in analysis:
        print(f'  Error: {analysis["error"]}')
        continue
    
    print(f'Total problems: {analysis["n_total"]}')
    print(f'CIF cases: {analysis["n_cif"]} ({analysis["cif_rate"]:.1%} of direct-correct)')
    
    if 'note' in analysis:
        print(f'Note: {analysis["note"]}')
        continue
    
    print(f'\nRecovery Rates:')
    print(f'{"Method":<15} {"Strength":<10} {"N":<6} {"Recovered":<10} {"Rate":<10}')
    print('-'*51)
    
    for recovery_name in RECOVERY_NAMES:
        if recovery_name in analysis['recovery_rates']:
            r = analysis['recovery_rates'][recovery_name]
            print(f'{recovery_name:<15} {r["strength"]:<10} {r["n_cases"]:<6} '
                  f'{r["n_recovered"]:<10} {r["recovery_rate"]:>7.1%}')

save_json(all_analyses, f'{SAVE_DIR_EXP}/results/analysis_recovery.json')

In [ ]:
# ============================================================
# CELL 11: VISUALIZATION
# ============================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors = {'sonnet4': '#5B8FF9', 'gpt4o': '#5AD8A6'}
model_labels = {'sonnet4': 'Claude Sonnet 4', 'gpt4o': 'GPT-4o'}
strength_colors = {'baseline': '#95A5A6', 'weak': '#F1C40F', 'medium': '#E67E22', 'strong': '#27AE60'}

# Plot 1: Recovery Rate by Method
ax1 = axes[0]
methods = ['none', 'verify', 'challenge', 'independent']
x = np.arange(len(methods))
width = 0.35

for i, model_key in enumerate(['sonnet4', 'gpt4o']):
    if model_key not in all_analyses:
        continue
    recovery_rates = [
        all_analyses[model_key].get('recovery_rates', {}).get(m, {}).get('recovery_rate', 0)
        for m in methods
    ]
    ax1.bar(x + i*width, recovery_rates, width,
            label=model_labels[model_key], color=colors[model_key])

ax1.set_ylabel('Recovery Rate', fontsize=12)
ax1.set_title('Recovery Rate by Prompt Method', fontsize=14)
ax1.set_xticks(x + width/2)
ax1.set_xticklabels(['None', 'Verify', 'Challenge', 'Independent'], rotation=15)
ax1.legend()
ax1.set_ylim(0, 1)

# Add strength indicators
for i, method in enumerate(methods):
    strength = RECOVERY_PROMPTS[method]['strength']
    ax1.annotate(f'({strength})', (i + width/2, -0.08),
                ha='center', fontsize=8, color=strength_colors[strength])

# Plot 2: Recovery Rate vs Strength (scatter)
ax2 = axes[1]
strength_map = {'baseline': 0, 'weak': 1, 'medium': 2, 'strong': 3}

for model_key in ['sonnet4', 'gpt4o']:
    if model_key not in all_analyses:
        continue
    
    strengths = []
    rates = []
    for method in methods:
        r = all_analyses[model_key].get('recovery_rates', {}).get(method, {})
        if 'recovery_rate' in r:
            strengths.append(strength_map[RECOVERY_PROMPTS[method]['strength']])
            rates.append(r['recovery_rate'])
    
    if strengths:
        ax2.scatter(strengths, rates, s=150, alpha=0.7,
                   label=model_labels[model_key], color=colors[model_key])
        # Trend line
        if len(strengths) >= 2:
            z = np.polyfit(strengths, rates, 1)
            p = np.poly1d(z)
            ax2.plot([0, 3], [p(0), p(3)], '--', color=colors[model_key], alpha=0.5)

ax2.set_xlabel('Prompt Strength', fontsize=12)
ax2.set_ylabel('Recovery Rate', fontsize=12)
ax2.set_title('Recovery Rate vs Prompt Strength', fontsize=14)
ax2.set_xticks([0, 1, 2, 3])
ax2.set_xticklabels(['Baseline', 'Weak', 'Medium', 'Strong'])
ax2.legend()
ax2.set_ylim(0, 1)

# Plot 3: CIF → Recovery Flow (Sankey-like)
ax3 = axes[2]

# Simple bar chart showing CIF rate vs best recovery rate
metrics = ['CIF Rate', 'Best Recovery']
x = np.arange(len(metrics))

for i, model_key in enumerate(['sonnet4', 'gpt4o']):
    if model_key not in all_analyses:
        continue
    
    cif_rate = all_analyses[model_key].get('cif_rate', 0)
    recovery_rates = all_analyses[model_key].get('recovery_rates', {})
    best_recovery = max([r.get('recovery_rate', 0) for r in recovery_rates.values()]) if recovery_rates else 0
    
    values = [cif_rate, best_recovery]
    ax3.bar(x + i*width, values, width,
            label=model_labels[model_key], color=colors[model_key])

ax3.set_ylabel('Rate', fontsize=12)
ax3.set_title('CIF Rate vs Best Recovery Rate', fontsize=14)
ax3.set_xticks(x + width/2)
ax3.set_xticklabels(metrics)
ax3.legend()
ax3.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR_EXP}/exp_A1_E10_recovery.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✓ Figure saved')

In [ ]:
# ============================================================
# CELL 12: FINAL SUMMARY
# ============================================================

summary = {
    'experiment_id': 'A1_E10',
    'experiment_name': 'Recovery Mechanisms',
    'date': EXPERIMENT_DATE,
    'hypothesis': 'Stronger recovery prompts lead to higher recovery from CIF',
    'recovery_methods': {k: v for k, v in RECOVERY_PROMPTS.items()},
    'n_problems': N_PROBLEMS,
    'models': list(MODELS.keys()),
    'results': all_analyses,
    'key_findings': []
}

for model_key in ['sonnet4', 'gpt4o']:
    if model_key not in all_analyses:
        continue
    
    analysis = all_analyses[model_key]
    recovery_rates = analysis.get('recovery_rates', {})
    
    if not recovery_rates:
        continue
    
    # Get recovery rate for each method
    rates_by_method = {m: recovery_rates.get(m, {}).get('recovery_rate', None) for m in RECOVERY_NAMES}
    
    # Find best method
    valid_rates = {m: r for m, r in rates_by_method.items() if r is not None}
    best_method = max(valid_rates.keys(), key=lambda m: valid_rates[m]) if valid_rates else None
    
    finding = {
        'model': model_key,
        'n_cif_cases': analysis.get('n_cif', 0),
        'cif_rate': analysis.get('cif_rate'),
        'recovery_rates': rates_by_method,
        'best_method': best_method,
        'best_recovery_rate': valid_rates.get(best_method) if best_method else None,
        'supports_hypothesis': (
            rates_by_method.get('independent', 0) > rates_by_method.get('verify', 0)
        ) if rates_by_method.get('independent') and rates_by_method.get('verify') else None
    }
    
    summary['key_findings'].append(finding)

save_json(summary, f'{SAVE_DIR_EXP}/results/exp_A1_E10_summary.json')

print('\n' + '='*60)
print('EXPERIMENT A1-E10 COMPLETE')
print('='*60)
print(f'\nResults saved to: {SAVE_DIR_EXP}')
print('\n' + '='*60)
print('KEY FINDINGS')
print('='*60)

for finding in summary['key_findings']:
    model_name = [n for n, c in MODELS.items() if c['short'] == finding['model']][0]
    print(f"\n{model_name}:")
    print(f"  CIF cases: {finding['n_cif_cases']}")
    print(f"  Recovery rates by method:")
    for method, rate in finding['recovery_rates'].items():
        if rate is not None:
            strength = RECOVERY_PROMPTS[method]['strength']
            print(f"    {method} ({strength}): {rate:.1%}")
    if finding['best_method']:
        print(f"  Best method: {finding['best_method']} ({finding['best_recovery_rate']:.1%})")
    print(f"  Supports hypothesis: {finding['supports_hypothesis']}")

print('\n' + '='*60)
print('INTERPRETATION')
print('='*60)
print('''
If hypothesis supported (stronger prompts → higher recovery):
  → Recovery prompts are an effective mitigation
  → "Independent" solving breaks contamination
  → Practical defense: Add verification steps

If not supported:
  → CIF is sticky/persistent
  → Simple prompts don't break contamination
  → Need stronger architectural interventions

Key metric: Best recovery rate
  - >50%: Recovery is possible
  - <20%: CIF is highly persistent
''')